In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

In [2]:
# list of .csv files in the directory
incart_csv = 'INCART 2-lead Arrhythmia Database' # this is the most relevant at the current time of writing this
arr_csv = 'MIT-BIH Arrhythmia Database'
supra_csv = 'MIT-BIH Supraventricular Arrhythmia Database'
scdh_csv = 'Sudden Cardiac Death Holter Database'

In [32]:
csv_path = os.path.join('..',
                        'raw_data', # root directory of the dataset folder
                        f'{incart_csv}.csv' # which csv are we looking at?
                        )
df = pd.read_csv(csv_path)

In [4]:
print(csv_path)

../raw_data/INCART 2-lead Arrhythmia Database.csv


In [5]:
rows = df.shape[0]
cols = df.shape[1]
print(f'The "{incart_csv}" has {rows} rows and {cols} columns.')

The "INCART 2-lead Arrhythmia Database" has 175729 rows and 34 columns.


In [6]:
df.head()

,record,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,I01,N,163,165,0.069610,-0.083281,0.614133,-0.392761,0.047159,15,...,-0.023370,14,3,23,6,-0.023370,-0.011650,0.082608,0.101373,-0.183387
1,I01,N,165,166,-0.097030,0.597254,-0.078704,-0.078704,-0.137781,3,...,0.081637,15,5,27,7,0.081637,0.102992,0.191225,0.217544,-0.068248
2,I01,N,166,102,0.109399,0.680528,-0.010649,-0.010649,-0.720620,6,...,-0.148539,33,13,52,6,-0.148539,-0.060620,0.081080,0.204400,0.335172
3,I01,VEB,102,231,0.176376,0.256431,-0.101098,-0.707525,-0.101098,4,...,0.046898,21,9,34,4,0.046898,0.083728,0.279512,0.526785,0.450969
4,I01,N,231,165,0.585577,0.607461,-0.083499,-0.083499,-0.167858,3,...,-0.112552,32,5,43,6,-0.112552,0.012989,0.091491,0.134004,0.265232


In [7]:
def load_data(folder_path, folder_name):
    """
    folder_path: string,
                 path of which folder has been saved on project
                 (folder_path = 'raw_data/')

    folder_name: string,
                 name of csv file inside of folder.

           Method:
               load_data('raw_data/Arythmia_monitor.csv') return (pandas.core.frame.DataFrame)
    """

    return pd.read_csv(folder_path + folder_name)

In [8]:
def clean():
    """
    Put descriptions please of what this function is doing
    """
    data = load_data('../raw_data/','INCART 2-lead Arrhythmia Database.csv')
    type_names = {
        'N': 'Normal',
        'SVEB': 'Supraventricular ectopic beat',
        'VEB': 'Ventricular ectopic beat',
        'F': 'Fusion beat',
        'Q': 'Unknown beat'
    }
    data = data.drop(columns=['record'])
    return data

In [9]:
def preprocess():
    """
    The preprocess of data before training
    """
    data = clean()

    X = data.drop('type', axis=1)
    y = data['type']
    type_mapping = {'N': 0, 'SVEB': 1, 'VEB': 2, 'F': 3, 'Q': 4}
    y = y.map(type_mapping)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    # Scaling the data before training
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #Resampling and rebalance the data
    smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    #Slicing to get light training memory
    subset_size = 10000
    X_train_subsample = X_train_resampled[:subset_size]
    y_train_subsample = y_train_resampled[:subset_size]

    return X_train_subsample, y_train_subsample, X_test, y_test

In [10]:
def initialize_model():
    """
    Good practice for any further modification
    """
    return RandomForestClassifier(random_state=101, n_estimators=50)

In [11]:
def main_stream():

    X_train, y_train, X_test, y_test = preprocess()
    model = initialize_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    #accuracy = accuracy_score(y_test, y_pred)
    return y_pred

In [12]:
main_output = main_stream()
main_output

array([0, 2, 0, ..., 0, 2, 0])

In [70]:
cols = str(df.columns[2:])
cleanX = cols.replace('-','')
cleanX

"Index(['0_preRR', '0_postRR', '0_pPeak', '0_tPeak', '0_rPeak', '0_sPeak',\n       '0_qPeak', '0_qrs_interval', '0_pq_interval', '0_qt_interval',\n       '0_st_interval', '0_qrs_morph0', '0_qrs_morph1', '0_qrs_morph2',\n       '0_qrs_morph3', '0_qrs_morph4', '1_preRR', '1_postRR', '1_pPeak',\n       '1_tPeak', '1_rPeak', '1_sPeak', '1_qPeak', '1_qrs_interval',\n       '1_pq_interval', '1_qt_interval', '1_st_interval', '1_qrs_morph0',\n       '1_qrs_morph1', '1_qrs_morph2', '1_qrs_morph3', '1_qrs_morph4'],\n      dtype='object')"

In [73]:
df.dtypes

record             object
type               object
0_pre-RR            int64
0_post-RR           int64
0_pPeak           float64
0_tPeak           float64
0_rPeak           float64
0_sPeak           float64
0_qPeak           float64
0_qrs_interval      int64
0_pq_interval       int64
0_qt_interval       int64
0_st_interval       int64
0_qrs_morph0      float64
0_qrs_morph1      float64
0_qrs_morph2      float64
0_qrs_morph3      float64
0_qrs_morph4      float64
1_pre-RR            int64
1_post-RR           int64
1_pPeak           float64
1_tPeak           float64
1_rPeak           float64
1_sPeak           float64
1_qPeak           float64
1_qrs_interval      int64
1_pq_interval       int64
1_qt_interval       int64
1_st_interval       int64
1_qrs_morph0      float64
1_qrs_morph1      float64
1_qrs_morph2      float64
1_qrs_morph3      float64
1_qrs_morph4      float64
dtype: object

In [58]:
print(df.dtypes.to_dict())


{'record': dtype('O'), 'type': dtype('O'), '0_pre-RR': dtype('int64'), '0_post-RR': dtype('int64'), '0_pPeak': dtype('float64'), '0_tPeak': dtype('float64'), '0_rPeak': dtype('float64'), '0_sPeak': dtype('float64'), '0_qPeak': dtype('float64'), '0_qrs_interval': dtype('int64'), '0_pq_interval': dtype('int64'), '0_qt_interval': dtype('int64'), '0_st_interval': dtype('int64'), '0_qrs_morph0': dtype('float64'), '0_qrs_morph1': dtype('float64'), '0_qrs_morph2': dtype('float64'), '0_qrs_morph3': dtype('float64'), '0_qrs_morph4': dtype('float64'), '1_pre-RR': dtype('int64'), '1_post-RR': dtype('int64'), '1_pPeak': dtype('float64'), '1_tPeak': dtype('float64'), '1_rPeak': dtype('float64'), '1_sPeak': dtype('float64'), '1_qPeak': dtype('float64'), '1_qrs_interval': dtype('int64'), '1_pq_interval': dtype('int64'), '1_qt_interval': dtype('int64'), '1_st_interval': dtype('int64'), '1_qrs_morph0': dtype('float64'), '1_qrs_morph1': dtype('float64'), '1_qrs_morph2': dtype('float64'), '1_qrs_morph3':

dtype('O')